# Approximate Nearest Neighbors:

# Image Recommendation System via Collaborative Filtering

# ***Please read the instructions very carefully***
This is a modified version of the previous question and requires you to use an artificial nearest neighbors library

We suggest you to use one of the following:
- [ScaNN](https://github.com/google-research/google-research/tree/master/scann)
- [FAISS](https://github.com/facebookresearch/faiss)
- [Annoy](https://github.com/spotify/annoy.git)

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   ⚠️ The Assignment will be evaluated automatically. Please adhere to taking proper inputs from `config.csv` file. You can change your `config.csv` file to experiment with your code. But at the end, make sure that your outputs are corresponding to input values in `config.csv`
5.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

## About the Dataset
Behance is a community art website where users showcase and discover creative work. Each user is able to “appreciate” (equivalent to a “like” on Instagram or a “react” on Facebook) an image, indicating that they like the image. It is in the website’s best interests to show users pictures that they would like, to keep them engaged for longer. For this question, given a set of pictures that a user has already appreciated, you have to show them a new picture that they would like based on what similar users appreciated.


<br><br>
**The dataset has information of 1 million appreciates of 63,497 users on 178,788 items. The file Behance appreciate 1M has a triplet in each line in the form of (user id, item id, unix timestamp).**

**Task: Take the inputs from the config.csv file and output the recommendations for a particular person**
- Collaborative Filtering is a way to predict items to the user based on the the
user’s history and the history of similar users. The similarity between users can be quantified by the number of images that both the users appreciated.
- The images appreciated by a similar user would be the most suitable images to show a user. Since we can find the similarity between any two users, we would be able to find the “nearest” neighbours of any user, allowing us to use a KNN-based algorithm to recommend new images to a user.
- Since people do not like seeing pictures that they have seen already. Make sure that you do not recommend pictures that a user has appreciated already.
- Output the final response will be saved in the file named ```config['output_file']```.


**Output file format:**
Populate the output file with images that the user has not seen of the k most
similar users, in descending order of their similarity. Each line in the output
file should be a duplet in the form of (item id, user id), where the user id is the
id of the kth similar user. The order of the images corresponding to the same
similar user would not matter. The output file would look something like this:
```
item_id_1_of_1st_similar_user 1st_most_similar_user_id
item_id_2_of_1st_similar_user 1st_most_similar_user_id
item_id_3_of_1st_similar_user 1st_most_similar_user_id
...
item_id_1_of_2nd_similar_user 2nd_most_similar_user_id
item_id_2_of_2nd_similar_user 2nd_most_similar_user_id
item_id_3_of_2nd_similar_user 2nd_most_similar_user_id
...
item_id_1_of_kth_similar_user kth_most_similar_user_id
item_id_2_of_kth_similar_user kth_most_similar_user_id
item_id_3_of_kth_similar_user kth_most_similar_user_id
```

You may use any other recommendation system that you wish to use. However,
evaluation script will score your submission by measuring the similarity between
users with the number of common images they appreciated.
The dataset was extracted using Behance’s API as a part of the paper
“Vista: A visually, socially, and temporally-aware model for artistic
recommendation, RecSys, 2016”. Check out this [Google Drive folder](https://drive.google.com/drive/folders/0B9Ck8jw-TZUEc3NlMjVXdDlPU1k?resourcekey=0-6_8ykn0o4fLc5fuTEm91xA) for
more information about the dataset.


Have fun! The users are waiting to see new pictures!

In [1]:
pip install scann

     |████████████████████████████████| 10.7 MB 7.9 MB/s 
     |████████████████████████████████| 462 kB 55.6 MB/s 


In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from numpy import array
from scipy.linalg import svd
from sklearn.decomposition import TruncatedSVD
from numpy import dot
from numpy.linalg import norm
import scann

### Import necessary libraries

In [3]:
# Config Generation Sample Code.
# ⚠️ Only for experimentation on your side.
# ⚠️ Should be commented during submission.

# df = pd.DataFrame(data=[{'id':276633,
#                   'k':5,
#                   'dataset_file':'./Behance_appreciate_1M',
#                   'output_file':'./output.txt'}])
# df.to_csv('config.csv')

In [4]:
config = pd.read_csv('config.csv').iloc[0]

In [5]:
config

Unnamed: 0                            0
id                               276633
k                                     5
dataset_file    ./Behance_appreciate_1M
output_file                ./output.txt
Name: 0, dtype: object

In [6]:
user = config['id']
k_value = config['k']

### Read the Data

In [7]:
with open(config['dataset_file'], 'r') as inFile:
  appreciate_data = inFile.readlines()

### Initialize a dictionary to store the item_ids that a user likes

### Go through each line of the input file and construct the user_likes dictionary

In [8]:
user_likes = dict()
users={}
items={}
u_ind=0
i_ind=0
user_row=[]
item_col=[]
matrix_val=[]
knn=[]

In [9]:
for line in appreciate_data:
  line = line.strip()
    
  user_id = int(line.split()[0])
  item_id = int(line.split()[1])

  if(user_id not in users):
    users[user_id]=u_ind
    u_ind+=1

  if(item_id not in items):
    items[item_id]=i_ind
    i_ind+=1
    
  user_row.append(users[user_id])
  item_col.append(items[item_id])
  matrix_val.append(1)
    

  if user_id not in user_likes:
    user_likes[user_id] = list()
    
  user_likes[user_id].append(item_id)

In [10]:
# print(len(users))
# print(len(items))

# print(len(user_row))
# print(len(item_col))
# print(len(matrix_val))
original_user_images=user_likes[user]
user_row=np.array(user_row)
item_col=np.array(item_col)
matrix_val=np.array(matrix_val)

csr = csr_matrix((matrix_val, (user_row, item_col)), shape=(len(users), len(items)))
# print(csr)
Singular_Value_Decomposition = TruncatedSVD(n_components=1000)

my_matrix =Singular_Value_Decomposition.fit_transform(csr)
searcher = scann.scann_ops_pybind.builder(my_matrix, 10, "dot_product").tree(num_leaves=1000, num_leaves_to_search=100, training_sample_size=len(my_matrix)).score_ah(2, anisotropic_quantization_threshold=0.2).reorder(100).build()
neighbors, distances = searcher.search(my_matrix[users[user]], final_num_neighbors=k_value+1)







### Use your choice of Approximate Nearest Neigbor after Collaborative Filtering to find nearest neighbors

In [11]:
knn=list(neighbors)
knn_final=[]
for this_user in users.keys():
  # print("hi")
  if(users[this_user] in knn and this_user!=user):
    knn_final.append(this_user)
# print(knn)
# print("hi")
print(knn_final)

[666917, 3628520, 2585131, 2492899, 3313136]


### Answer the following questions:

#### Q1. **Explain how your choice of library works**

*your solution here*
I have selected Scann scalable nearest neighbors.
scann is an efficient vector similarity search algorithm.There are various ways by which we can calculate similars vectors to a given vectors.
like cosine , eucledian distances etc.
Scann uses Anisotropic Vector Quantization to better calculate the inner products.
Benchmarks test performed by ann-benchmark.com scann outperforms various other libraries .

#### Q2. **Compare your choice of library with vanilla KNN.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*
Vanilla KNN: Vanilla knn or simply knn is an algorithm to find nearest neighbors to a given point.
there are various senerio where we have to calculate distance between various vectors.
what vanilla knn do is it plot each vectors in a d dimention vectors space and then calculate the distance between each and every point using various parameters and properties, then after calculating the distances we select the minimum or top k distance vectors and these will be our k nearest neihbors.
Time complexity of vanilla knn is really high since it calculates the distance of vectors with every other vectors so it is really slow.

Scann : Scann is approximate nearest neighbors algo. it uses vector quantization.
scann is a madeup of various things like score aware loss , anisotropic loss and product quantization.
also by SIMD in register lookup tables search speed will be accelerated and hence time complexity will be improved drastically.


#### Q3. **Compare your choice of library with implementation of ScaNN, faiss and annoy.**
***Hint: Include Time Complexity, and explain the tradeoff with recall***

*your solution here*
Scann is different from faiss and annoy because :- 

Annoy : Annoy stands for approximate nearest neighbors oh Yeah! Annoy divides the whole vector space into various partition.first of all it selects to random points and left side will be one region and right side will be other region. it form a tree kind of structure and follows top down traversal.
two points are close to each other then they will be close to each other in the tree also.

Faiss : Nearest neighbors is hard to achieve due to curse of dimensionality.
so instead of bruteforce what we do in faiss is we transform the matrix and reducec the search area.
faiss uses an indexing approach followed by 2 quantization operations.
Also faiss is a c++ library so it a by defualt fast.



### Open the output file to write all the lines to the file

In [12]:
outFile = open(config['output_file'], 'w')

for n_user in knn_final:
    # user_id = list(user_likes.keys())[n_user]
    user_id=n_user
    # print("userid is ",user_id)
    for item_id in user_likes[user_id]:
        if(item_id not in original_user_images):
            outFile.write(str(item_id) + ' ' + str(user_id) + '\n')

outFile.close()